### Imports

In [ ]:
import pandas as pd
from pandas import DataFrame as DF
import matplotlib.pyplot as plt

from core.plt_utils import plt_3d_df
from watea.fleet_watea_wise_perfs import fleet_charge_energy_points_df
from watea.watea_constants import *
from watea.processed_watea_ts import processed_ts_of
from watea.watea_fleet_info import fleet_info_df
from watea.watea_perfs import compute_perfs

### Toutes les distributions

In [ ]:
def preprocess_dist_df(raw_dist_df: DF) -> DF:
    return (
        raw_dist_df
        .assign(soc=lambda df: df.index.get_level_values(2))
        .assign(temp=lambda df: df.index.get_level_values(1))
        .assign(odo=lambda df: df.index.get_level_values(0))
        .drop(columns=["duration"])
        .query("energy_added < 500")
        .eval("power = energy_added / sec_duration")
    )
df:DF = fleet_charge_energy_points_df.pipe(preprocess_dist_df)

plt_3d_df(df, "soc", "odo"    ,"energy_added"    ,color="odo"    ,save_path=PATH_TO_FLEET_3D_DISTRIBUTION    ,size=4.5)

### Seul la distribution du "haut" est presente parmi toutes les ranges d'odometer

### Zoom sur la distribution la plus presente

In [ ]:
def crop_most_commoon_dist(raw_dist_df:DF) -> DF:
    return (
        raw_dist_df
        .xs(15.0    ,level=1)
        .query("energy_added > 300 & sec_duration < 900 & temp < 35")
    )
most_common_distribution_df = crop_most_commoon_dist(df)

plt_3d_df(most_common_distribution_df, "soc", "odo", "energy_added", color="power", save_path=PATH_TO_FLEET_3D_DISTRIBUTION, size=4.5)

### Nettoyage

In [ ]:
def clean_distribution(dist_df: DF) -> DF:
    cleaned_df_mask = (
        dist_df
        .groupby(level=[0, 1])["energy_added"]
        .transform(lambda energy_grp: energy_grp.gt(energy_grp.quantile(0.1)))
    )
    return (
        dist_df
        # .loc[cleaned_df_mask]
        .query("power < 4 & power > 1.5")
    )

cleaned_df = clean_distribution(most_common_distribution_df)

plt_3d_df(cleaned_df, "soc", "odo", "energy_added", color="power", save_path=PATH_TO_FLEET_3D_DISTRIBUTION, size=4.5)

### Influence de la puissace dans un groupe odometre+temp 

In [ ]:
XS_IDX = (0, 15.0)

fleet_charge_energy_points_xs = (
    fleet_charge_energy_points_df
    .query("energy_added > 325 & energy_added < 500 & sec_duration < 900 & temp < 35 & power < 4 & power > 3")
    # .loc[:, "energy_added"]
    .xs(XS_IDX)
    # .dropna()
)
plt_3d_df(cleaned_df, "soc", "power", "energy_added", color="power", save_path=PATH_TO_FLEET_3D_DISTRIBUTION, size=4.5)


### Comparaison entre la distribution du permier et du denier interval d'odometre

In [ ]:
def get_odo_grp_xs(dist_df:DF, grp_idx:int) -> DF:
    keys = dist_df.index.get_level_values(level=0).unique()
    first_odo_grp = dist_df.xs(key=keys[0], level=0).sort_index()
    last_odo_grp = dist_df.xs(key=keys[grp_idx], level=0).sort_index()
    
    return first_odo_grp, keys[0], last_odo_grp, keys[grp_idx]

def compute_median_energy_dist_per_soc(distrib_grp: DF) -> pd.Series:
    return (
        distrib_grp
        .loc[:, "energy_added"]
        .groupby(level=0)
        .agg("median")
        .rolling(4, center=True)
        .median()
        .mask(lambda series: series < series.cummax(), pd.NA)
        .interpolate(method="index")
        .rolling(10, center=True)
        .mean()
    )

first_fleet_dist_xs, first_odo_value, last_fleet_dist_xs, last_odo_value = get_odo_grp_xs(cleaned_df, -1)

def plt_two_dist_xs(first_xs, first_odo_value, second_xs, second_odo_value):
    compute_median_energy_dist_per_soc(first_xs).plot.line(color="green", label=ODO_RANGE_FORMAT_STR(first_odo_value))
    compute_median_energy_dist_per_soc(second_xs).plot.line(color="red", label=ODO_RANGE_FORMAT_STR(second_odo_value))
    plt.scatter(first_xs["soc"], first_xs["energy_added"], color="green", alpha=0.075)
    plt.scatter(second_xs["soc"], second_xs["energy_added"], color="red", alpha=0.15)
    plt.legend()
    plt.show()

plt_two_dist_xs(first_fleet_dist_xs, first_odo_value, last_fleet_dist_xs, last_odo_value)

### Obtention d'une distribution d'un vehicule

In [ ]:
biggest_odo_idx = fleet_info_df.query("has_power_during_charge")["max_odo"].argmax()
id = fleet_info_df.query("has_power_during_charge")["id"].iat[biggest_odo_idx]
vehicle_df = processed_ts_of(id)
perfs = compute_perfs(vehicle_df, id, force_update=True)

vehicle_energy_dist_df:DF = perfs["charge_energy_points"].pipe(preprocess_dist_df)

plt_3d_df(
    vehicle_energy_dist_df.query("energy_added > 300 & energy_added < 500 & sec_duration < 900 & temp < 35 & power < 4 & power > 1.5"),
    "soc", 
    "power", 
    "energy_added", 
    color="power", 
    size=4.5,  
    opacity=0.8
)

### Comparaison avec la distribution odo [0, 3k]

In [ ]:
def clean_single_vehicle_dist_df(dist_df: DF) -> DF:
    cleaned_df_mask = (
        dist_df
        .groupby([
            dist_df["odo"],
            dist_df["soc"].floordiv(2).mul(2)
            ])["energy_added"]
        .transform(lambda energy_grp: energy_grp.gt(energy_grp.quantile(0.45)))
    )
    return (
        dist_df
        # .mask(cleaned_df_mask, pd.NA)
        .query("power < 4 & power > 1.5")
    )

sub_vehicle_dist_df = (
    vehicle_energy_dist_df
    .pipe(crop_most_commoon_dist)
    .pipe(clean_single_vehicle_dist_df)
)

first_vehicle_dist_xs, first_vhehicle_odo_xs_value, last_vehicle_dist_xs, vhehicle_odo_last_value = get_odo_grp_xs(sub_vehicle_dist_df, -1)

plt_two_dist_xs(first_fleet_dist_xs, first_odo_value, last_vehicle_dist_xs, vhehicle_odo_last_value)

### calcul du soh

In [ ]:
distribution_ratios:DF = (
    DF({
        "vehicle_dist_median": compute_median_energy_dist_per_soc(last_vehicle_dist_xs),
        "fleet_dist_median": compute_median_energy_dist_per_soc(first_fleet_dist_xs),
    })
    .eval("dist_ratio = vehicle_dist_median / fleet_dist_median")
)

plt.scatter(first_fleet_dist_xs["soc"], first_fleet_dist_xs["energy_added"], color="green", alpha=0.075)
plt.scatter(last_vehicle_dist_xs["soc"], last_vehicle_dist_xs["energy_added"], color="red", alpha=0.15)
distribution_ratios["fleet_dist_median"].plot.line(color="green", label=ODO_RANGE_FORMAT_STR(first_odo_value))
distribution_ratios["vehicle_dist_median"].plot.line(color="red", label=ODO_RANGE_FORMAT_STR(vhehicle_odo_last_value))
ratio_ax = plt.gca().twinx()
distribution_ratios["dist_ratio"].plot.line(color="violet", label=f"ratio over soc, mean:{distribution_ratios['dist_ratio'].mean():.2f}")
plt.legend()
plt.show()